In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, dayofweek, date_format
import pandas as pd
from zipfile import ZipFile
from pyspark.sql.types import * 

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
input_data = 'data/'
output_data = 'data/output-data/'

## Processing song data

#### Songs table

In [50]:
# with ZipFile('data/song-data.zip', 'r') as zip_ref:
#     zip_ref.extractall('data/local-song-data')

In [4]:
song_data = input_data + 'local-song-data/song_data/*/*/*/*.json'

In [5]:
df = spark.read.json(song_data)
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [6]:
# extract columns to create songs table
songs_table = df.select('song_id', 'title', 'artist_id', 'year', 'duration').drop_duplicates()
songs_table.show(5)

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOGOSOV12AF72A285E|   ¿Dónde va Chichi?|ARGUVEV1187B98BA17|1997|313.12934|
|SOTTDKS12AB018D69B|It Wont Be Christmas|ARMBR4Y1187B9990EB|   0|241.47546|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|2004|207.77751|
|SOIAZJW12AB01853F1|          Pink World|AR8ZCNI1187B9A069B|1984|269.81832|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|2005|186.48771|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows



In [7]:
 # write songs table to parquet files partitioned by year and artist
songs_table.write.parquet(output_data + 'songs', mode='overwrite', partitionBy=['year', 'artist_id'])

#### Artists table

In [8]:
artists_table = df.select('artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude').drop_duplicates()
artists_table.show(5)

+------------------+---------------+---------------+---------------+----------------+
|         artist_id|    artist_name|artist_location|artist_latitude|artist_longitude|
+------------------+---------------+---------------+---------------+----------------+
|AR3JMC51187B9AE49D|Backstreet Boys|    Orlando, FL|       28.53823|       -81.37739|
|AR0IAWL1187B9A96D0|   Danilo Perez|         Panama|         8.4177|       -80.11278|
|ARWB3G61187FB49404|    Steve Morse| Hamilton, Ohio|           null|            null|
|AR47JEX1187B995D81|   SUE THOMPSON|     Nevada, MO|       37.83721|       -94.35868|
|ARHHO3O1187B989413|      Bob Azzam|               |           null|            null|
+------------------+---------------+---------------+---------------+----------------+
only showing top 5 rows



In [13]:
# write artists table to parquet files
artists_table.write.parquet(output_data + 'artists', mode='overwrite')

## Processing log data

In [ ]:
# with ZipFile('data/log-data.zip', 'r') as zip_ref:
#     zip_ref.extractall('data/local-log-data')

In [14]:
# get filepath to log data file
log_data = input_data + 'local-log-data/*.json'

In [15]:
# read log data file
df =  spark.read.json(log_data)

In [16]:
df.printSchema()
df.show(5)

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            lo

In [17]:
df.count()

8056

In [18]:
# filter by actions for song plays
df = df.filter(df['page'] == 'NextSong')

In [19]:
df.count()

6820

#### Users table

In [20]:
# extract columns for users table    
users_table = df.select('userId', 'firstName', 'lastName', 'gender', 'level').drop_duplicates()
users_table.show(3)

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    57|Katherine|     Gay|     F| free|
|    84|  Shakira|    Hunt|     F| free|
|    22|     Sean|  Wilson|     F| free|
+------+---------+--------+------+-----+
only showing top 3 rows



In [21]:
# write users table to parquet files
users_table.write.parquet(output_data + 'users', mode='overwrite')

#### Time table

In [22]:
df.select('ts').show(3)

+-------------+
|           ts|
+-------------+
|1542241826796|
|1542242481796|
|1542242741796|
+-------------+
only showing top 3 rows



In [23]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.utcfromtimestamp(int(x) / 1000), TimestampType())
df = df.withColumn('start_time', get_timestamp('ts'))

In [24]:
df.select('start_time').show(3)

+--------------------+
|          start_time|
+--------------------+
|2018-11-15 00:30:...|
|2018-11-15 00:41:...|
|2018-11-15 00:45:...|
+--------------------+
only showing top 3 rows



In [38]:
# create datetime column from original timestamp column
get_datetime = udf(lambda x: datetime.utcfromtimestamp(int(x) / 1000), DateType())
df = df.withColumn('start_time_dtf', get_datetime('ts'))

In [41]:
df.select('start_time_dtf').show(5)

+--------------+
|start_time_dtf|
+--------------+
|    2018-11-15|
|    2018-11-15|
|    2018-11-15|
|    2018-11-15|
|    2018-11-15|
+--------------+
only showing top 5 rows



In [25]:
# extract columns to create time table
time_table = df.withColumn('hour', hour('start_time'))\
    .withColumn('day', dayofmonth('start_time'))\
    .withColumn('week', weekofyear('start_time'))\
    .withColumn('month', month('start_time'))\
    .withColumn('year', year('start_time'))\
    .withColumn('weekday', dayofweek('start_time'))\
    .select('ts', 'start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday').dropDuplicates()
    
time_table.show(3)                           

+-------------+--------------------+----+---+----+-----+----+-------+
|           ts|          start_time|hour|day|week|month|year|weekday|
+-------------+--------------------+----+---+----+-----+----+-------+
|1542279962796|2018-11-15 11:06:...|  11| 15|  46|   11|2018|      5|
|1542299805796|2018-11-15 16:36:...|  16| 15|  46|   11|2018|      5|
|1542765178796|2018-11-21 01:52:...|   1| 21|  47|   11|2018|      4|
+-------------+--------------------+----+---+----+-----+----+-------+
only showing top 3 rows



In [27]:
# write time table to parquet files partitioned by year and month
time_table.write.parquet(output_data + 'time', mode='overwrite', partitionBy=['year', 'month'])

#### Songplay table

In [28]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)



In [30]:
# read in song data to use for songplays table
song_df = spark.read.format('json').load(input_data + 'local-song-data/song_data/*/*/*').drop_duplicates()

In [31]:
song_df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [32]:
song_df.show(3)

+------------------+---------------+---------------+----------------+-----------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|artist_location|artist_longitude|artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+---------------+----------------+-----------+---------+---------+------------------+--------------------+----+
|ARPFHN61187FB575F6|       41.88415|    Chicago, IL|       -87.63241|Lupe Fiasco|279.97995|        1|SOWQTQZ12A58A7B63E|Streets On Fire (...|   0|
|AR1Y2PT1187FB5B9CE|       27.94017|        Brandon|       -82.32547|John Wesley|484.62322|        1|SOLLHMX12AB01846DC|   The Emperor Falls|   0|
|AR7G5I41187FB4CE6C|           null|London, England|            null|   Adam Ant|233.40363|        1|SONHOTT12A8C13493C|     Something Girls|1982|
+------------------+---------------+---------------+----------------+-----------+---------+---------+-----------------

In [33]:
# extract columns from joined song and log datasets to create songplays table 
songplays_table = df.join(song_df, df.song == song_df.title, how='inner') \
    .select(monotonically_increasing_id().alias('songplay_id'), col('start_time'),col('userId').alias('user_id') \
    , 'level', 'song_id', 'artist_id', col('sessionId').alias('session_id'), 'location', col('userAgent').alias('user_agent'))

In [34]:
# add year and month columns to use to partition by in the next step
songplays_table = songplays_table.join(time_table, songplays_table.start_time == time_table.start_time, how='inner')\
                        .select('songplay_id', songplays_table.start_time, 'user_id', 'level', 'song_id', 'artist_id' \
                                , 'session_id', 'location', 'user_agent', 'year', 'month').drop_duplicates()

In [35]:
songplays_table.show(3)

+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|songplay_id|          start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|year|month|
+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|          0|2018-11-21 21:56:...|     15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|2018|   11|
|          2|2018-11-19 09:14:...|     24| paid|SOGDBUF12A8C140FAA|AR558FS1187FB45658|       672|Lake Havasu City-...|"Mozilla/5.0 (Win...|2018|   11|
|          3|2018-11-27 22:35:...|     80| paid|SOGDBUF12A8C140FAA|AR558FS1187FB45658|       992|Portland-South Po...|"Mozilla/5.0 (Mac...|2018|   11|
+-----------+--------------------+-------+-----+------------------+------------------+--------

In [36]:
songplays_table.printSchema()

root
 |-- songplay_id: long (nullable = false)
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



In [37]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.parquet(output_data + 'songplays', mode='overwrite', partitionBy=['year', 'month'])